<a href="https://colab.research.google.com/github/floriandendorfer/demand-estimation/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>